# Imports and Setups

In [4]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import matplotlib.pyplot as plt
import numpy as np
import Dataset

### Setup the `device` variable.

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Download and Prepare Dataset

In [6]:

dataset = Dataset.flowDataset()
batch_size = 50
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)



In [7]:
for imgs, labels in test_loader:
    print("Batch of images has shape: ",imgs.shape)
    print("Batch of labels has shape: ", labels.shape)

Batch of images has shape:  torch.Size([50, 3, 200, 1000])
Batch of labels has shape:  torch.Size([50, 1])
Batch of images has shape:  torch.Size([50, 3, 200, 1000])
Batch of labels has shape:  torch.Size([50, 1])
Batch of images has shape:  torch.Size([50, 3, 200, 1000])
Batch of labels has shape:  torch.Size([50, 1])
Batch of images has shape:  torch.Size([50, 3, 200, 1000])
Batch of labels has shape:  torch.Size([50, 1])
Batch of images has shape:  torch.Size([50, 3, 200, 1000])
Batch of labels has shape:  torch.Size([50, 1])
Batch of images has shape:  torch.Size([50, 3, 200, 1000])
Batch of labels has shape:  torch.Size([50, 1])
Batch of images has shape:  torch.Size([50, 3, 200, 1000])
Batch of labels has shape:  torch.Size([50, 1])
Batch of images has shape:  torch.Size([50, 3, 200, 1000])
Batch of labels has shape:  torch.Size([50, 1])
Batch of images has shape:  torch.Size([50, 3, 200, 1000])
Batch of labels has shape:  torch.Size([50, 1])
Batch of images has shape:  torch.Siz

In [8]:
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

# Define Model Architecture

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()             # input: 1000*200*3
        self.conv1 = nn.Conv2d(3, 32, 3, 1,1)     # output: 1000*200*32
        self.maxpool1 = nn.MaxPool2d(2,2)       # output: 500*100*32
        self.conv2 = nn.Conv2d(32, 64, 3, 1,1)    # output: 500*100*64 
        self.maxpool2 = nn.MaxPool2d(2,2)       # output: 250*50*64
        self.conv3 = nn.Conv2d(64, 128, 3, 1,1)   # output: 250*50*128
        self.maxpool3 = nn.MaxPool2d(2,2)       # output: 125*25*128
        self.conv4 = nn.Conv2d(128, 128, 3, 1,1)  # output: 125*25*128
        self.maxpool4 = nn.MaxPool2d(5,5)       # output: 25*5*128
        self.fc1 = nn.Linear(16000, 7744)       # in:15488 out:7744
        self.fc2 = nn.Linear(7744, 128)         # in:7744 out:128
        self.fc3 = nn.Linear(128, 10)           # in:128  out:10

    def forward(self, x):
        ## Conv 1st Block
        x = self.conv1(x)
        x = F.relu(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.maxpool3(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.maxpool4(x)


        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

# Training Loop

In [10]:
def train(model, device, train_loader, optimizer, epoch, steps_per_epoch=20):
  # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.train()
  train_total = 0
  train_correct = 0

  # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
  for batch_idx, (data, target) in enumerate(train_loader, start=0):
    if batch_idx > steps_per_epoch:
      break
    # Load the input features and labels from the training dataset
    data, target = data.to(device), target.to(device)
    
    # Reset the gradients to 0 for all learnable weight parameters
    optimizer.zero_grad()
    
    # Forward pass: Pass image data from training dataset, make predictions about class image belongs to (0-9 in this case)
    output = model(data)
    
    # Define our loss function, and compute the loss
    loss = F.nll_loss(output, target)

    scores, predictions = torch.max(output.data, 1)
    train_total += target.size(0)
    train_correct += int(sum(predictions == target))
            
    # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
    loss.backward()
    
    # Update the neural network weights
    optimizer.step()

  acc = round((train_correct / train_total) * 100, 2)
  print('Epoch [{}], Loss: {}, Accuracy: {}'.format(epoch, loss.item(), acc), end='')
  

# Testing Loop

In [11]:
def test(model, device, test_loader, classes):
  # Switch model to evaluation mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.eval()
  test_loss = 0
  test_total = 0
  test_correct = 0

  example_images = []
  with torch.no_grad():
      for data, target in test_loader:
          # Load the input features and labels from the test dataset
          data, target = data.to(device), target.to(device)
          
          # Make predictions: Pass image data from test dataset, make predictions about class image belongs to (0-9 in this case)
          output = model(data)
          
          # Compute the loss sum up batch loss
          test_loss += F.nll_loss(output, target, reduction='sum').item()
          
          scores, predictions = torch.max(output.data, 1)
          test_total += target.size(0)
          test_correct += int(sum(predictions == target))
          
          # WandB – Log images in your test dataset automatically, along with predicted and true labels by passing pytorch tensors with image data into wandb.Image
          # example_images.append(wandb.Image(
          #     data[0], caption="Pred: {} Truth: {}".format(classes[pred[0].item()], classes[target[0]])))
  acc = round((test_correct / test_total) * 100, 2)
  print(' Test_loss: {}, Test_accuracy: {}'.format(test_loss/test_total, acc))


# Train

In [12]:
net = Net().to(device)
print(net)

optimizer = optim.Adam(net.parameters())

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool4): MaxPool2d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16000, out_features=7744, bias=True)
  (fc2): Linear(in_features=7744, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [13]:

for epoch in range(10):
  train(net, device, train_loader, optimizer, epoch)
  test(net, device, test_loader, classes)

print('Finished Training')

C:\Users\reese\miniconda3\envs\final-year-project\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


RuntimeError: CUDA out of memory. Tried to allocate 154.00 MiB (GPU 0; 8.00 GiB total capacity; 6.17 GiB already allocated; 0 bytes free; 6.23 GiB reserved in total by PyTorch)